In [1]:
#https://cloud.tencent.com/developer/article/1454904
#https://zhuanlan.zhihu.com/p/82850698
#https://blog.csdn.net/weixin_42598761/article/details/104592171

In [2]:
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Lambda, Bidirectional, LSTM, Dense
from keras_bert import load_trained_model_from_checkpoint
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from datetime import datetime
from collections import Counter
import keras.callbacks
import re
import codecs
import time

Using TensorFlow backend.


In [3]:
from ckiptagger import WS, POS, NER
ckip_path = r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\data'
ws = WS(ckip_path, disable_cuda=False) #斷詞
pos = POS(ckip_path, disable_cuda=False) #詞性標注
ner = NER(ckip_path, disable_cuda=False) #實體辨識

In [4]:
import os
import codecs
import pandas as pd
import numpy as np

bert_dir = r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\bert'
config_path = os.path.join(bert_dir, 'bert_config.json')
checkpoint_path = os.path.join(bert_dir, 'bert_model.ckpt')
dict_path = os.path.join(bert_dir, 'vocab.txt')

# 0. 載入資料

In [91]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_0620.csv')
data = data[data["status"]=="ok"].drop(["url","context","raw_content","status", "content_status"],axis = 1)
data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)
data['name'] = data['name'].apply(lambda x: eval(x))

In [92]:
data['content'] = data['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data['content'] = data['content'].apply(lambda x: x.replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data['content'] = data['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))

In [93]:
train = data

In [94]:
data2 = pd.read_csv(r'C:\Users\rocker\extra data\人肉爬蟲_20200615.csv', encoding='cp950')
data2['name'] = data2['name'].apply(lambda x: eval(x))
data2['content'] = data2['content'].apply(lambda x: re.sub('<[^>]*>|【[^】]*】|（[^）]*）|〔[^〕]*〕', '', x))
data2['content'] = data2['content'].apply(lambda x: x.replace('記者', '＜')
                                                   .replace('報導', '＞')
                                                   .replace('▲', '')\
                                                   .replace('。　', '。')\
                                                   .replace('', '')\
                                                   .replace('.', '')\
                                                   .replace(' ', '')\
                                                   .replace('“', '「')\
                                                   .replace('”', '」'))
data2['content'] = data2['content'].apply(lambda x: re.sub('＜[^＞]*＞', '', x))
test = data2
test = test[['news_id', 'name', 'content', 'aml_label']]

In [ ]:
train, test =  train_test_split(data, test_size = 0.2, random_state = 0)

In [9]:
# 把大於512的train以逗點單雙句分開，預設為aml_label=1的都分，aml_label=0的中取frac比率分，藉此調整整體正反樣本比例
def split_train(data,frac=1):
    data_more_split = pd.DataFrame()
    
    data_0 = data[data['aml_label'] == 0].sample(frac=frac, replace=False)
    data_1 = data[data['aml_label'] == 1]
    data = data_0.append(data_1)
    data_nochange_0 = data[~data['news_id'].isin(data_0['news_id'])]
    
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content'].split('，')
            even = '，'.join([s for i, s in enumerate(s) if i % 2 == 0])
            odd = '，'.join([s for i, s in enumerate(s) if i % 2 == 1])
            contents = [even, odd]
            
            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content'].split('，')
            first = '，'.join([s for i, s in enumerate(s) if i % 3 == 0])
            second = '，'.join([s for i, s in enumerate(s) if i % 3 == 1])
            third = '，'.join([s for i, s in enumerate(s) if i % 3 == 2])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)
    data_more_split = data_nochange_0.append(data_more_split)
    return data_more_split

In [ ]:
#分句
train = train.drop(['name'], axis=1)
data_less = train[train['content'].str.len() <= 512]
data_more = train[train['content'].str.len() > 512]
data_more_split = split_train(data_more, 1)
train = train.append(data_more_split)
train = train.reset_index().drop(['index'], axis=1)

In [ ]:
train

In [ ]:
content = '曾找英國天后JessieJ、謝金燕、張惠妹等大牌藝人造勢的「富南斯」集團'
input_id, segment_id = tokenizer.encode(content, max_len=maxlen)
mask_input = [transfer(i) for i in input_id]
prediction = model.predict([[input_id], [segment_id], [mask_input]])
prediction

In [126]:
content = '檢察官其他人名依法判徐正憲貪污罪'
input_id, segment_id = tokenizer.encode(content, max_len=maxlen)
mask_input = [transfer(i) for i in input_id]
prediction = model2.predict([[input_id], [segment_id], [mask_input]])
prediction

array([[0.49339]], dtype=float32)

# 1. 建立 aml 模型

In [10]:
def create_tokenizer(dict_path):
    
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
            
    return token_dict

def transfer(i):
    
    if i != 0:
        return 1
    else:
        return 0

def encoded(tokenizer, data, maxlen):
    
    x, y, z = [], [], []
    if 'content' in data.columns:
        for content in data['content']:
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = [transfer(i) for i in x1]
            x.append(x1)
            y.append(x2)
            z.append(x3)
    elif 'Sentence' in data.columns:
        for content in data['Sentence']:
            x1, x2 = tokenizer.encode(content, max_len=maxlen)
            x3 = [transfer(i) for i in x1]
            x.append(x1)
            y.append(x2)
            z.append(x3)
            
    return x, y, z

In [11]:
token_dict = create_tokenizer(dict_path)
tokenizer = Tokenizer(token_dict)

In [12]:
maxlen = 256
batch_size = 8
epochs = 3

In [13]:
label = np.asarray(train['aml_label'])

In [14]:
input_id, segment_id, mask_input = encoded(tokenizer, train, maxlen)

In [15]:
def bert_LSTM_model():
    
    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen)
    sequence_output = model.layers[-9].output
    #sequence_output = Lambda(lambda x: x[:, 0])(sequence_output)
    sequence_output = Bidirectional(LSTM(128, return_sequences=False))(sequence_output)
    output = Dense(1, activation='sigmoid')(sequence_output)
    model = Model(model.input, output)
    
    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    
    return model

model = bert_LSTM_model()

In [16]:
# 載入模型
model.load_weights('aml_model_weight.h5')

In [ ]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [ ]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )

In [ ]:
model.save_weights('aml_model_weight_512.h5')

# 2. 建立 NER 模型
### transfer_NER 要跑 1.5 小時，訓練模型要跑 17mins/epoch，所以load weight就好

In [17]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content}, index=[66]), ignore_index=True)
    
    return data_more_split

In [18]:
maxlen_ner = 512
batch_size = 8
epochs = 3
input_shape = (maxlen_ner, )

In [19]:
# 取得 NER input array (陳水扁貪汙 -> CKIP(陳水扁) -> 1 2 2 0 0)
def transfer_NER(data, tokenizer, maxlen):
    people_list = []
    first_name = ['申', '龔', '馮', '昌', '劉', '習', '陽', '顧', '鍾', '胡', '許', '魏',    '傅', '季', '扶', '柳', '狄', '焦', '封', '李', '羿', '刁', '和', '邴',    '陸', '王', '杜', '能', '侯', '伍', '平', '竺', '樂', '繆', '欒', '湛',    '道', '花', '賴', '浦', '萬', '章', '宮', '勾', '邵', '印', '夏', '杭',    '溥', '左', '池', '公', '閻', '符', '奚', '臧', '羅', '空', '璩', '巴',    '酈', '范', '談', '金', '顏', '慎', '郭', '僪', '聞', '車', '闞', '相',    '童', '雙', '方', '莊', '容', '姚', '田', '薛', '閔', '翟', '簡',    '蔚', '茹', '淩', '戴', '余', '鞏', '房', '富', '牛', '饒', '計', '居',    '後', '舒', '席', '翁', '祝', '鬱', '訾', '隆', '匡', '弘', '曆', '範',    '越', '趙', '卻', '岑', '隗', '冷', '張', '山', '松', '柯', '嵇', '韓',    '蕭', '褚', '殳', '滕', '滿', '洪', '荀', '庾', '廖', '盧', '危', '竇',    '曾', '郎', '遊', '穀', '慕', '禹', '凌', '廉', '邢', '梁', '葉',    '郝', '終', '齊', '藺', '曹', '全', '高', '樊', '史', '桂', '廣', '段',    '江', '餘', '袁', '弓', '牧', '魚', '儲', '尚', '逄', '尹', '通', '懷',    '皮', '何', '倪', '包', '晁', '涂', '蓬', '屠', '巫', '須', '巢', '卞',    '楊', '成', '孟', '楚', '呂', '古', '毋', '伊', '賁', '喻', '糜',    '蔔', '艾', '藍', '龐', '諸', '別', '任', '管', '冀', '壽', '惠', '梅',    '孫', '從', '康', '常', '駱', '鞠', '沈', '黨', '沙', '鳳', '郁', '邊',    '仰', '溫', '路', '逮', '賀', '雷', '鈄', '明', '裴', '滑', '毛', '費',    '關', '時', '步', '麴', '裘', '蒲', '司', '查', '錢', '盛', '霍', '鮑',    '彭', '龍', '沃', '單', '勞', '秋', '祖', '殷', '茅', '敖', '郗', '石',    '鐘', '嚴', '畢', '燕', '姜', '經', '程', '厙', '柏', '汪', '婁', '胥',    '聶', '邰', '桑', '辛', '扈', '穆', '仲', '紅', '項', '師', '桓', '黃',    '堵', '貢', '詹', '朱', '蔡', '戈', '于', '甄', '束', '屈', '索', '晏',    '阮', '魯', '虞', '歐', '濮', '俞', '黎', '文', '應', '姬', '貝', '籍',    '莘', '戚', '鄭', '郜', '景', '宋', '宗', '昝', '卓', '蒯', '馬', '顔',    '蘇', '衛', '東', '瞿', '蒼', '莫', '邱', '潘', '家', '林', '芮', '麻',    '元', '武', '強', '鈕', '陳', '井', '於', '游', '耿', '柴', '荊', '韶',    '易', '宿', '施', '鹹', '秦', '班', '甯', '汲', '酆', '暴', '尤',    '祿', '苗', '權', '仇', '都', '羊', '榮', '陶', '支', '賈', '白', '葛',    '暨', '解', '靳', '伏', '唐', '華', '吉', '融', '豐', '安', '衡', '那',    '闕', '俄', '盍', '鄔', '蒙', '利', '鄂', '謝', '宓', '湯', '喬', '孔',    '養', '紀', '幹', '牟', '連', '宰', '蔣', '雍', '益', '寇', '祁', '熊',    '崔', '丁', '薊', '譚', '吳', '烏', '周', '農', '徐', '充', '向', '宦',    '董', '甘', '冉', '韋', '米', '鄒', '鄧', '戎', '水']
    label = np.zeros([len(data), maxlen])
    
    # 用CKIP抓出每個新聞的名字
    for index, (_, row) in enumerate(data.iterrows()):
        
        if index % 100 == 0:
            print(index)
        
        token = tokenizer.tokenize(row['content'][0:maxlen])

        if len(token) > maxlen:
            token = token[0:maxlen-1]
            token.append('[SEP]')

        y = np.zeros([maxlen])
        content = ''.join(token)
        
        #CKIP
        word_sentence_list = ws([content],
                    sentence_segmentation=True,
                    segment_delimiter_set={'?', '？', '!', '！', '。', ',', '，', ';', ':', '、'})
        pos_sentence_list = pos(word_sentence_list)
        entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
        
        people = [people for people in list(entity_sentence_list[0]) if (people[2] == 'PERSON') & (people[1] < maxlen)]
        people = [people for people in people if ((len(people[3]) < 5) & (people[3][0] in first_name)) | ((len(people[3]) >= 5) & ('#' not in people[3]))]
        people.sort()
        people_list.append(people)
        
        #轉換成input array
        j = 0
        for person in people: 
            for i, _ in enumerate(token):        
                if token[i:i+len(person[3])] == list(person[3]):
                    if len(person) == 1:
                        y[i+j] = 1
                        token = token[i+1:]
                        j = i+j+1
                        break

                    y[i+j] = 1
                    y[i+j+1:i+j+len(person[3])] = 2
                    token = token[i+len(person[3]):]
                    j = i+j+len(person[3])
                    break
        label[index, :] = y
    
    #用不到 people_list，只是檢查用
    return people_list, label.reshape([label.shape[0], label.shape[1], 1])

In [ ]:
#分句
test_ner = aml_highrisk.drop(['name'], axis=1)
data_less = test_ner[test_ner['content'].str.len() <= 512]
data_more = test_ner[test_ner['content'].str.len() > 512]
data_more_split = split_content(data_more)
test_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

In [ ]:
people_list, label = transfer_NER(train_ner, tokenizer, maxlen=maxlen_ner)

In [ ]:
input_id, segment_id, mask_input = encoded(tokenizer, train_ner, maxlen=maxlen_ner)

In [20]:
def bert_BiLSTM_CRF_model():
    
    ner_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_ner)
    bert_output = ner_model.layers[-9].output
    X = Lambda(lambda x: x[:, 0: input_shape[0]])(bert_output)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    #X = TimeDistributed(Dense(len(y_token_dict), activation='relu'))(X)
    output = CRF(3, sparse_target = True)(X)    
    ner_model = Model(ner_model.input, output)
    
    for layer in ner_model.layers:
        layer.trainable = False
    ner_model.layers[-1].trainable = True
    ner_model.layers[-2].trainable = True
    
    return ner_model

In [21]:
ner_model = bert_BiLSTM_CRF_model()

In [22]:
# transfer_NER 要跑 1.5 小時，訓練模型要跑 17mins/epoch，所以load weight就好
ner_model.load_weights('ner_model_weight.h5')

In [ ]:
total_steps, warmup_steps = calc_train_steps(
    num_example=data.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [ ]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_crf_accuracy', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [ ]:
ner_model.compile(optimizer=optimizer,
                  loss=crf_loss,
                  metrics=[crf_accuracy])

ner_model.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          validation_split=0.1,
          callbacks=callback_list)

# Model 2

In [108]:
train = train[train['aml_label'] == 1]

In [109]:
maxlen_sentences = 256
batch_size = 8
epochs = 2

In [110]:
# 不用跑
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content_model2(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label']}, index=[66]), ignore_index=True)
    
    return data_more_split

In [111]:
def predict_sentences_model2(test, people_list, stick, tokenizer=tokenizer, maxlen=maxlen_sentences):
    
    AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
    aml_highrisk = np.asarray(test['content'])
    news_ids = np.asarray(test['news_id'])
    people_list = list(test['people_list'])

    
    for k, (news_id, y_news) in enumerate(zip(news_ids ,aml_highrisk)): 
        #用，。？切分句子
        y_news = y_news.replace('。','=。')
        y_news = y_news.replace('，','*，')
        y_news = y_news.replace('？','+？')
        
        news = re.split('，|。|？', y_news)
        
        news = [news.replace('=','。') for news in news]
        news = [news.replace('*','，') for news in news]
        news = [news.replace('+','？') for news in news]
#############################################################################################################
        for i in range(len(people_list[k])):
            # 找出人名存在的 news index
            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                                
            x = [people for people in people_list[k] if people_list[k][i] not in people]

            name = [name for name in people_list[k] if name != people_list[k][i]]
            name = [name for name in name if ((len(name)<3) & (name[0] != people_list[k][i][0])) | (len(name)>=3)]
            name.sort(reverse=True)

            #if name != []:
            #    text = [re.sub('|'.join(name), '其他人名', news) for news in news]
            #else:
            #    text = news

            for j in index:

                mid = news[j]

                if j == 0:
                    if len(news) != 1:
                        end = news[j+1]  

                        if True in [people in news[j+1] for people in x]:
                            sentences = mid
                        else:
                            sentences = mid + end
                    elif len(news) == 1:
                        sentences = mid

                elif j+1 == len(news):
                    start = news[j-1]

                    if True in [people in news[j-1] for people in x]:
                        sentences = mid
                    else:
                        sentences = start + mid

                else:
                    end = news[j+1]
                    start = news[j-1]                    

                    if '。' in start:
                        start = ''
                    elif '。' in mid:
                        end = ''                 

                    if (True not in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = start + mid + end
                    elif (True not in [people in news[j-1] for people in x]) & (True in [people in news[j+1] for people in x]):
                        sentences = start + mid
                    elif (True in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = mid + end
                    else:
                        sentences = mid      

                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
                #break  
                
    #若 stick==True 則把多筆同姓名句子以逗點合併 (效果不好)
    if stick:
        AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()
    
    
    # 把指向同一人的姓名改成一樣（陳男 -> 陳水扁），若指向多人則不改（陳男 -> 陳致中、陳水扁）
    # 將預測不完整的名字回填（王音 -> 王音之）
    name_list = []
    for ids in AML['news_id'].unique():
        full_name = [name for name in AML[(AML['news_id'] == ids)]['Name']]
        full_3name = [name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) == 3]
        
        a = Counter([name[0] for name in full_3name])
        keep = [k for k,v in a.items() if v == 1]
        full_3name_filter = [name for name in full_3name if name[0] in keep]
        name_dict = dict((name[0], name) for name in full_3name_filter)   # ex: {'陳' : '陳水扁'}

        name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # ex: {'王音': '王音之'}
        
        for name in full_name:
            if (name[0] in name_dict.keys()) & (len(name) == 1):
                name_list.append(name_dict.get(name[0]))
            elif (name[0] in name_dict.keys()) & (len(name) == 2) & (name[-1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                                  '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                                  '和', '揆', '要', '再', '董', '涉', '母',\
                                                                                  '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                                  '趁', '仔', '依', '氏', '父']):
                name_list.append(name_dict.get(name[0]))
            elif (name in name_dict_2.keys()) & (len(name) == 2):
                name_list.append(name_dict_2.get(name))
            else:
                name_list.append(name)
                
    
    # 排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
    AML['Name'] = name_list
    AML = AML.drop_duplicates()
    AML = AML[AML['Name'].apply(lambda x: (len(x) > 1) )]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                     '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                     '和', '揆', '要', '再', '董', '涉', '母',\
                                                                     '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                     '趁', '仔', '依', '氏', '父']))]
    
    return AML

In [112]:
# 取得名字 (預測結果為onehot的狀態)
def get_name(input_id, y_pred):
    
    label_list = []
    word_dict = {v: k for k, v in token_dict.items()}
    
    for input_data, y in zip(input_id, y_pred):
        people_index = ''.join([str(a) for a in list(y)])
        j = 0
        name_list = []
        split_index = re.findall('[12]2*', people_index)
        name = ''.join([word_dict.get(input_data[index]) for index, value in enumerate(y) if value != 0])
        
        # [UNK], [PAD]會被算成 5 個字元，避免轉換成文字的index因長度不同對不上，故用 1 個字元的其他符號替代
        # 王春甡 -> 王春[UNK] -> 王春?
        name = name.replace('[UNK]','?')
        name = name.replace('[PAD]','!')
        
        for i in split_index:
            name_list.append(name[0+j:len(i)+j])
            j = len(i) + j
            
        name_list = [name for name in name_list]
        label_list.append(list(set(name_list)))
    
    return label_list

In [113]:
# 2. 將超過 512 的句子以句點拆成多句分段預測
train_ner = train.drop(['name'], axis=1)
data_less = train_ner[train_ner['content'].str.len() <= 512]
data_more = train_ner[train_ner['content'].str.len() > 512]
data_more_split = split_content_model2(data_more)
train_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

# 3. NER 預測人名
input_id, segment_id, mask_input = encoded(tokenizer, train_ner, maxlen=maxlen_ner)
prediction = ner_model.predict([input_id, segment_id, mask_input])
y_pred = np.argmax(prediction, axis=-1)
people_list = get_name(input_id, y_pred)


# 4. 將拆開的句子組合回去
train_ner['people_list'] = people_list
content = train_ner[['news_id', 'content', 'aml_label']]
content = content.groupby(['news_id', 'aml_label'])['content'].apply(lambda x : '。'.join(x)).reset_index()
people = train_ner[['news_id', 'aml_label', 'people_list']]
people = people.groupby(['news_id', 'aml_label'])['people_list'].agg(sum).reset_index()
people['people_list'] = [list(set(people)) for people in people['people_list']]
train_ner = pd.merge(content, people, on=['news_id', 'aml_label'], how='left')

# 5. 將 [UNK], [PAD] 轉換回來 (王春? -> 王春甡)
for _, row in train_ner.iterrows():
    for i, name in enumerate(row['people_list']):
        if ('?' in name) | ('!' in name):
            reexp = name.replace('?', '.').replace('!', '.')
            row['people_list'][i] = re.search(reexp, row['content']).group()
            
AML = predict_sentences_model2(train_ner, list(train_ner['people_list']), tokenizer=tokenizer, maxlen=maxlen_sentences, stick=False)

AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()

train_name = train.drop(['content'], axis=1)
s = train_name.apply(lambda x: pd.Series(x['name']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Name'
train_name = train_name.drop('name', axis=1).join(s)

aml_train = pd.merge(AML, train_name, on=['news_id', 'Name'], how='left')
aml_train = aml_train.fillna(0)

In [114]:
aml_train

,news_id,Name,Sentence,aml_label
0,18,王派宏,自稱房產幽默大師的王派宏，涉吸金捲款25億落跑！他自稱炒房專家，，被害的投資人懷疑，王派宏已...,1.0
1,38,李威儀,以250萬元行賄時任內政部都市計畫委員會委員李威儀與其學生藍秀琪；經法院12年審理，，201...,1.0
2,38,王桂霜,12年前爆發的花蓮縣壽豐鄉鯉魚潭風景區的「綠湖國際大飯店」開發弊案，業者王桂霜為讓計畫案順利...,1.0
3,38,藍秀琪,以250萬元行賄時任內政部都市計畫委員會委員李威儀與其學生藍秀琪；經法院12年審理，，201...,1.0
4,41,陳廷智,「台灣第一家」涉嫌將工業用碳酸鎂摻入椒鹽粉等產品中出售，創辦人陳廷智之子、總經理陳星佑被高等...,0.0
...,...,...,...,...
1056,4980,王光遠,曾被週刊爆料涉嫌假冒華固建設高層，並涉土地投資詐騙糾紛的前國防部政治作戰總隊總隊長王光遠將軍...,0.0
1057,4981,邱佳亮,桃園市議員邱佳亮被控貪污案，一審遭判12年6月，，依據二審台灣高等法院裁定書指出，邱佳亮聲請...,1.0
1058,4996,楊天生,郭是長億集團董事長楊天生的姪女婿，曾任台灣省警務處督察、台中縣霧峰警分局長，,0.0
1059,4996,蔡茂寅,長生透過台大教授蔡茂寅申請仲裁，事後高鐵局仲裁案敗訴，,0.0


In [116]:
extra = aml_train[aml_train['Sentence'].str.contains('檢察官')]
aml_train = aml_train.append(extra).append(extra).append(extra)

In [117]:
label = np.asarray(aml_train['aml_label'])

In [118]:
input_id, segment_id, mask_input = encoded(tokenizer, aml_train, maxlen_sentences)

In [119]:
def bert_LSTM_model2():
    
    model2 = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen_sentences)
    sequence_output = model2.layers[-9].output
    #sequence_output = Lambda(lambda x: x[:, 0])(sequence_output)
    sequence_output = Bidirectional(LSTM(128, return_sequences=False))(sequence_output)
    output = Dense(1, activation='sigmoid')(sequence_output)
    model2 = Model(model2.input, output)
    
    for layer in model2.layers:
        layer.trainable = False
    model2.layers[-1].trainable = True
    model2.layers[-2].trainable = True
    
    return model2

model2 = bert_LSTM_model2()

In [120]:
total_steps, warmup_steps = calc_train_steps(
    num_example=train.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [121]:
callback_list = [
                 keras.callbacks.EarlyStopping(monitor='val_acc', patience=1)
                 #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
                ]

In [122]:
model2.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

model2.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          #validation_split=0.1,
          #callbacks=callback_list
         )

Epoch 1/2
1370/1370 [==============================] - 66s 48ms/step - loss: 0.3625 - acc: 0.8431
Epoch 2/2
1370/1370 [==============================] - 64s 47ms/step - loss: 0.2788 - acc: 0.8825


In [ ]:
model2.save_weights('aml_model_2.h5')

In [30]:
model2.load_weights('192_aml_model_stick.h5')

In [ ]:
model2.load_weights('aml_model_2.h5')

In [ ]:
model.summary()

# 3. 預測

In [123]:
def predict_aml(model, test, aml_threshold):
    
    #第一階段預測，大於aml_threshold者為疑似aml文章
    input_id, segment_id, mask_input = encoded(tokenizer, test, maxlen=maxlen)
    prediction = model.predict([input_id, segment_id, mask_input])
    prediction[prediction >= aml_threshold] = 1
    prediction[prediction < aml_threshold] = 0
    
    return prediction

In [39]:
def predict_sentences(test, people_list, stick, threshold, tokenizer=tokenizer, maxlen=maxlen_sentences):
    
    time = datetime.now()
    
#    AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
#    aml_highrisk = np.asarray(test['content'][test['prediction'] == 1])
#    news_ids = np.asarray(test['news_id'][test['prediction'] == 1])
#    people_list = list(test['people_list'][test['prediction'] == 1])
    
    AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
    aml_highrisk = np.asarray(test['content'])
    news_ids = np.asarray(test['news_id'])
    people_list = list(test['people_list'])
    
    for k, (news_id, y_news) in enumerate(zip(news_ids ,aml_highrisk)): 
        #用，。？切分句子
        y_news = y_news.replace('。','=。')
        y_news = y_news.replace('，','*，')
        y_news = y_news.replace('？','+？')
        
        news = re.split('，|。|？', y_news)
        
        news = [news.replace('=','。') for news in news]
        news = [news.replace('*','，') for news in news]
        news = [news.replace('+','？') for news in news]
#############################################################################################################
        for i in range(len(people_list[k])):
            # 找出人名存在的 news index
            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                                
            x = [people for people in people_list[k] if people_list[k][i] not in people]

            name = [name for name in people_list[k] if name != people_list[k][i]]
            name = [name for name in name if ((len(name)<3) & (name[0] != people_list[k][i][0])) | (len(name)>=3)]
            name.sort(reverse=True)

            #if name != []:
            #    text = [re.sub('|'.join(name), '其他人名', news) for news in news]
            #else:
            #    text = news

            for j in index:

                mid = news[j]
                
                if name != []:
                    mid = re.sub('|'.join(name), '其他人名', mid)

                if j == 0:
                    if len(news) != 1:
                        end = news[j+1]  

                        if True in [people in news[j+1] for people in x]:
                            sentences = mid
                        else:
                            sentences = mid + end
                    elif len(news) == 1:
                        sentences = mid

                elif j+1 == len(news):
                    start = news[j-1]

                    if True in [people in news[j-1] for people in x]:
                        sentences = mid
                    else:
                        sentences = start + mid

                else:
                    end = news[j+1]
                    start = news[j-1]                    

                    if '。' in start:
                        start = ''
                    elif '。' in mid:
                        end = ''                 

                    if (True not in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = start + mid + end
                    elif (True not in [people in news[j-1] for people in x]) & (True in [people in news[j+1] for people in x]):
                        sentences = start + mid
                    elif (True in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                        sentences = mid + end
                    else:
                        sentences = mid      

                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
                #break  
#############################################################################################################        
                
#        for i in range(len(people_list[k])):
#            # 找出人名存在的 news index
#            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                           
#            
#            ##3
#            x = [people for people in people_list[k] if people_list[k][i] not in people]
#            
#            for j in index:
#                
#                name = [name for name in people_list[k] if name != people_list[k][i]]
#                name = [name for name in name if (len(name)>=1) & (name[0] != people_list[k][i][0])]
#                name.sort(reverse=True)
#                new_news = re.sub('|'.join(name), '其他人名', news[j])
#                
#                if j == 0:
#                    if len(news) != 1:
#                        new_news2 = re.sub('|'.join(name), '其他人名', news[j+1])  
#                        sentences = new_news + new_news2
#                    elif len(news) == 1:
#                        sentences = new_news
#
#                elif j+1 == len(news):
#                    new_news3 = re.sub('|'.join(name), '其他人名', news[j-1]) 
#                    sentences = new_news3 + '，' + new_news
#            
#                else:
#                    new_news2 = re.sub('|'.join(name), '其他人名', news[j+1]) 
#                    new_news3 = re.sub('|'.join(name), '其他人名', news[j-1])                    
#
#                    
#                    if '。' in new_news3:
#                        new_news3 = ''
#                    elif '。' in new_news:
#                        new_news2 = '' 
#                        
#                    sentences = new_news3 + new_news + new_news2
#                
#                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
#                break
#############################################################################################################        
        
       
#        for i in range(len(people_list[k])):
#            #找出人名存在的news index
#            index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                        
#                   
#            #取前後一句
#            for j in index:
#                
#                name = [name for name in people_list[k] if name != people_list[k][i]]
#                name = [name for name in name if (len(name)>=1) & (name[0] != people_list[k][i][0])]
#                name.sort(reverse=True)
#                new_news1 = re.sub('|'.join(name), '其他人名', news[j])
#                
#                if j == 0:
#                    new_news2 = re.sub('|'.join(name), '其他人名', news[j+1])
#                    sentences = new_news1 + new_news2
#                elif j+1 == len(news):
#                    new_news0 = re.sub('|'.join(name), '其他人名', news[j-1])
#                    sentences = new_news0 + new_news1
#                else:
#                    new_news0 = re.sub('|'.join(name), '其他人名', news[j-1])
#                    new_news2 = re.sub('|'.join(name), '其他人名', news[j+1])
#                    sentences = new_news0 + new_news1 + new_news2
#                    
#                AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
#                #break
#############################################################################################################                
                
    print('1.提取句子', datetime.now() - time)
    time = datetime.now()
    
    #若 stick==True 則把多筆同姓名句子以逗點合併 (效果不好)
    if stick:
        AML = AML.groupby(['news_id', 'Name'])['Sentence'].apply('，'.join).reset_index()

    
    # 把指向同一人的姓名改成一樣（陳男 -> 陳水扁），若指向多人則不改（陳男 -> 陳致中、陳水扁）
    # 將預測不完整的名字回填（王音 -> 王音之）
    name_list = []
    for ids in AML['news_id'].unique():
        full_name = [name for name in AML[(AML['news_id'] == ids)]['Name']]
        full_3name = [name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) == 3]
        
        a = Counter([name[0] for name in full_3name])
        keep = [k for k,v in a.items() if v == 1]
        full_3name_filter = [name for name in full_3name if name[0] in keep]
        name_dict = dict((name[0], name) for name in full_3name_filter)   # ex: {'陳' : '陳水扁'}

        name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # ex: {'王音': '王音之'}
        
        for name in full_name:
            if (name[0] in name_dict.keys()) & (len(name) == 1):
                name_list.append(name_dict.get(name[0]))
            elif (name[0] in name_dict.keys()) & (len(name) == 2) & (name[-1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                                  '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                                  '和', '揆', '要', '再', '董', '涉', '母',\
                                                                                  '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                                  '趁', '仔', '依', '氏', '父']):
                name_list.append(name_dict.get(name[0]))
            elif (name in name_dict_2.keys()) & (len(name) == 2):
                name_list.append(name_dict_2.get(name))
            else:
                name_list.append(name)
                
    print('2.整理名字', datetime.now() - time)
    time = datetime.now()
    
    # 排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
    AML['Name'] = name_list
    AML = AML.drop_duplicates()
    AML = AML[AML['Name'].apply(lambda x: (len(x) > 1) )]
    AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                     '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                     '和', '揆', '要', '再', '董', '涉', '母',\
                                                                     '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                     '趁', '仔', '依', '氏', '父']))]
    
    
    print('3.刪除名字', datetime.now() - time)
    time = datetime.now()
    
    # 第二階段 預測句子
    input_id, segment_id, mask_input = encoded(tokenizer, AML, maxlen=maxlen)
    prediction = model2.predict([input_id, segment_id, mask_input])
    AML['prediction'] = prediction
    
    
    # 同一人只要有一筆資料大於閥值（max），則預測為 aml 人物；若新聞中無人大於閥值，則為非 aml 新聞
    AML['prediction'] = AML['prediction'].apply(lambda x: 0 if x < threshold else 1)
    AML = AML.groupby(['news_id', 'Name'])['prediction'].max().reset_index()
    AML = AML[AML['prediction'] == 1]
    AML = AML.groupby(['news_id','prediction'])['Name'].apply(list).reset_index()
    
    print('4.預測名字', datetime.now() - time)
    
    return AML

In [33]:
def f1_score(a, b):
    
    if (len(a) != 0) & (len(b) != 0):
        recall = float(len(set(a) & set(b)) / len(a))
        pecision = float(len(set(a) & set(b)) / len(b))
        score = 2 / (np.reciprocal(recall) + np.reciprocal(pecision))
        return score
    elif (len(a) == 0) & (len(b) == 0):
        return 1
    else:
        return 0

# 4. 網格搜索

### 4-1 [NER法]

In [34]:
# 把大於512的新聞以句點分段 (bert最多只能吃512)
def split_content(data):
    data_more_split = pd.DataFrame()
    for i, row in data.iterrows():
        if (len(row['content']) > 512) & (len(row['content']) <= 1024):

            s = row['content']
            s_split = [(i, abs(len(s)//2 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left = min(s_split, key=lambda x: x[1])[0]
            first = "。".join([s_split[i][2] for i in range(idx_left)])
            second = "。".join([s_split[i][2] for i in range(idx_left, len(s_split))])    
            contents = [first, second]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label'], 'prediction':row['prediction']}, index=[66]), ignore_index=True)

        elif len(row['content']) > 1024:

            s = row['content']
            s_split1 = [(i, abs(len(s)//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            s_split2 = [(i, abs(len(s)*2//3 - s.find(x)), x) for i, x in enumerate(s.split('。'))]
            idx_left1 = min(s_split1, key=lambda x: x[1])[0]
            idx_left2 = min(s_split2, key=lambda x: x[1])[0]
            first = "。".join([s_split1[i][2] for i in range(idx_left1)])
            second = "。".join([s_split1[i][2] for i in range(idx_left1, idx_left2)])
            third = "。".join([s_split1[i][2] for i in range(idx_left2, len(s_split1))])
            contents = [first, second, third]

            for content in contents:
                data_more_split = data_more_split.append(pd.DataFrame({'news_id':row['news_id'], 'content':content, 'aml_label':row['aml_label'], 'prediction':row['prediction']}, index=[66]), ignore_index=True)
    
    return data_more_split

In [35]:
def GridSearch_ner(df, model, test, aml_threshold, stick, threshold):
    
    time = datetime.now()
    
    # 1. 預測是否疑似 aml
    prediction = predict_aml(model, test=test, aml_threshold=aml_threshold)
    test['prediction'] = prediction
    aml_highrisk = test[test['prediction'] == 1]
    
    # 2. 將超過 512 的句子以句點拆成多句分段預測
    test_ner = aml_highrisk.drop(['name'], axis=1)
    data_less = test_ner[test_ner['content'].str.len() <= 512]
    data_more = test_ner[test_ner['content'].str.len() > 512]
    data_more_split = split_content(data_more)
    test_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)
    
    # 3. NER 預測人名
    input_id, segment_id, mask_input = encoded(tokenizer, test_ner, maxlen=maxlen_ner)
    prediction = ner_model.predict([input_id, segment_id, mask_input])
    y_pred = np.argmax(prediction, axis=-1)
    people_list = get_name(input_id, y_pred)
    
    
    # 4. 將拆開的句子組合回去
    test_ner['people_list'] = people_list
    content = test_ner[['news_id', 'content', 'aml_label', 'prediction']]
    content = content.groupby(['news_id', 'aml_label', 'prediction'])['content'].apply(lambda x : '。'.join(x)).reset_index()
    people = test_ner[['news_id', 'aml_label', 'prediction', 'people_list']]
    people = people.groupby(['news_id', 'aml_label', 'prediction'])['people_list'].agg(sum).reset_index()
    people['people_list'] = [list(set(people)) for people in people['people_list']]
    test_ner = pd.merge(content, people, on=['news_id', 'aml_label', 'prediction'], how='left')
    
    # 5. 將 [UNK], [PAD] 轉換回來 (王春? -> 王春甡)
    for _, row in test_ner.iterrows():
        for i, name in enumerate(row['people_list']):
            if ('?' in name) | ('!' in name):
                reexp = name.replace('?', '.').replace('!', '.')
                row['people_list'][i] = re.search(reexp, row['content']).group()
    
    
    print('0.CKIP', datetime.now() - time)
    
    # 6. 判斷名字前後句使是否為 aml
    AML = predict_sentences(test_ner, list(test_ner['people_list']), tokenizer=tokenizer, maxlen=maxlen_sentences, stick=stick, threshold=threshold)
    
    test_prediction = pd.merge(test, AML[['news_id', 'Name']], on='news_id', how='left')
    test_prediction['Name'] = test_prediction['Name'].apply(lambda x: x if isinstance(x, list) else [])
    test_prediction['text_prediction'] = test_prediction['Name'].apply(lambda x: 0 if x == [] else 1)
    test_prediction = test_prediction.drop(['content'],axis = 1)
    test_prediction.columns = ['news_id', 'name', 'label', 'AML_prediction', 'Name_prediction', 'text_prediction']
    
    # 7. 算分數
    score = []
    for i in range(len(test_prediction)):
        temp = f1_score(test_prediction['name'][i], test_prediction['Name_prediction'][i])
        score.append(temp)
        
    test_prediction['f1_score'] = score    
    total_score = sum(score)
    aml_score = sum(test_prediction[test_prediction['label'] == 1]['f1_score'])    
    
    df = df.append(pd.DataFrame([[aml_threshold, threshold, stick, total_score, aml_score]], columns=df.columns))
    
    print('aml_threshold =', aml_threshold, 'stick =', stick, 'threshold =', threshold, 'total_score =', total_score, 'aml_score =', aml_score)

    return df

In [36]:
df = pd.DataFrame(columns=['aml_threshold', 'threshold', 'stick', 'total_score', 'aml_score'])

In [37]:
len(test[test['aml_label'] ==1])

67

In [124]:
for aml_threshold in [0.4]:
    for threshold in [0.3, 0.4, 0.5, 0.6]:
        for stick in [False]:        
            df = GridSearch_ner(df, model, test, aml_threshold=aml_threshold, stick=stick, threshold=threshold)

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0.CKIP 0:00:05.628005
1.提取句子 0:00:01.220765
2.整理名字 0:00:00.048870
3.刪除名字 0:00:00.003991
4.預測名字 0:00:11.278925
aml_threshold = 0.4 stick = False threshold = 0.3 total_score = 76.20158730158731 aml_score = 56.201587301587296
0.CKIP 0:00:05.559844
1.提取句子 0:00:01.221296
2.整理名字 0:00:00.048871
3.刪除名字 0:00:00.003988
4.預測名字 0:00:09.574399
aml_threshold = 0.4 stick = False threshold = 0.4 total_score = 77.11111111111111 aml_score = 57.11111111111109
0.CKIP 0:00:05.558675
1.提取句子 0:00:01.221761
2.整理名字 0:00:00.048869
3.刪除名字 0:00:00.002992
4.預測名字 0:00:09.753418
aml_threshold = 0.4 stick = False threshold = 0.5 total_score = 77.53333333333333 aml_score = 57.53333333333332
0.CKIP 0:00:05.538762
1.提取句子 0:00:01.222733
2.整理名字 0:00:00.048870
3.刪除名字 0:00:00.002994
4.預測名字 0:00:09.682104
aml_threshold = 0.4 stick = False threshold = 0.6 total_score = 75.85317460317461 aml_score = 55.8531746031746


C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in reciprocal
  


In [40]:
for aml_threshold in [0.4]:
    for threshold in [0.3, 0.4, 0.5, 0.6]:
        for stick in [False]:        
            df = GridSearch_ner(df, model, test, aml_threshold=aml_threshold, stick=stick, threshold=threshold)

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0.CKIP 0:00:05.766550
1.提取句子 0:00:01.270661
2.整理名字 0:00:00.048872
3.刪除名字 0:00:00.003987
4.預測名字 0:00:09.546463
aml_threshold = 0.4 stick = False threshold = 0.3 total_score = 75.98650793650795 aml_score = 55.986507936507934


C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in reciprocal
  


0.CKIP 0:00:05.544135
1.提取句子 0:00:01.225749
2.整理名字 0:00:00.048872
3.刪除名字 0:00:00.002994
4.預測名字 0:00:09.644201
aml_threshold = 0.4 stick = False threshold = 0.4 total_score = 75.39603174603175 aml_score = 55.396031746031746
0.CKIP 0:00:05.556143
1.提取句子 0:00:01.228686
2.整理名字 0:00:00.048870
3.刪除名字 0:00:00.004022
4.預測名字 0:00:09.636243
aml_threshold = 0.4 stick = False threshold = 0.5 total_score = 72.11984126984126 aml_score = 52.119841269841274
0.CKIP 0:00:05.611538
1.提取句子 0:00:01.231711
2.整理名字 0:00:00.048870
3.刪除名字 0:00:00.003989


KeyboardInterrupt: 

In [77]:
for aml_threshold in [0.4]:
    for threshold in [0.3]:
        for stick in [False]:        
            df = GridSearch_ner(df, model, test, aml_threshold=aml_threshold, stick=stick, threshold=threshold)

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0.CKIP 0:00:05.520736
1.提取句子 0:00:01.229748
2.整理名字 0:00:00.048903
3.刪除名字 0:00:00.003992
4.預測名字 0:00:09.511551
aml_threshold = 0.4 stick = False threshold = 0.3 total_score = 75.86825396825398 aml_score = 55.86825396825395


In [ ]:
for aml_threshold in [0.4]:
    for threshold in [0.4]:
        for stick in [False]:        
            df = GridSearch_ner(df, model, test, aml_threshold=aml_threshold, stick=stick, threshold=threshold)

# 5. NER, CKIP比較 (待整理)

In [78]:
# 預測是否疑似aml
prediction = predict_aml(model, test, aml_threshold=0.4)
test['prediction'] = prediction
aml_highrisk = test[test['prediction'] == 1]

# 將超過512的句子拆成多句分段預測
test_ner = aml_highrisk.drop(['name'], axis=1)
data_less = test_ner[test_ner['content'].str.len() <= 512]
data_more = test_ner[test_ner['content'].str.len() > 512]
data_more_split = split_content(data_more)
test_ner = data_less.append(data_more_split).reset_index().drop(['index'], axis=1)

# 預測人名
input_id, segment_id, mask_input = encoded(tokenizer, test_ner, maxlen=maxlen_ner)
prediction = ner_model.predict([input_id, segment_id, mask_input])
y_pred = np.argmax(prediction, axis=-1)
people_list = get_name(input_id, y_pred)

# 將拆開的句子組合回去
test_ner['people_list'] = people_list
content = test_ner[['news_id', 'content', 'aml_label', 'prediction']]
content = content.groupby(['news_id', 'aml_label', 'prediction'])['content'].apply(lambda x : '。'.join(x)).reset_index()
people = test_ner[['news_id', 'aml_label', 'prediction', 'people_list']]
people = people.groupby(['news_id', 'aml_label', 'prediction'])['people_list'].agg(sum).reset_index()
people['people_list'] = [list(set(people)) for people in people['people_list']]
test_ner = pd.merge(content, people, on=['news_id', 'aml_label', 'prediction'], how='left')

# 將 [UNK], [PAD] 轉換回來
for _, row in test_ner.iterrows():
    for i, name in enumerate(row['people_list']):
        if ('?' in name) | ('!' in name):
            reexp = name.replace('?', '.').replace('!', '.')
            row['people_list'][i] = re.search(reexp, row['content']).group()

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
AML = pd.DataFrame(columns=['news_id', 'Name', 'Sentence'])
aml_highrisk = np.asarray(test_ner['content'][test_ner['prediction'] == 1])
news_ids = np.asarray(test_ner['news_id'][test_ner['prediction'] == 1])
people_list = list(test_ner['people_list'][test_ner['prediction'] == 1])



for k, (news_id, y_news) in enumerate(zip(news_ids ,aml_highrisk)): 
    #用，。？切分句子
    y_news = y_news.replace('。','=。')
    y_news = y_news.replace('，','*，')
    y_news = y_news.replace('？','+？')

    news = re.split('，|。|？', y_news)

    news = [news.replace('=','。') for news in news]
    news = [news.replace('*','，') for news in news]
    news = [news.replace('+','？') for news in news]
#############################################################################################################
    for i in range(len(people_list[k])):
        # 找出人名存在的 news index
        index = [index for index, _ in enumerate(news) if people_list[k][i] in _]                                
        x = [people for people in people_list[k] if people_list[k][i] not in people]

        name = [name for name in people_list[k] if name != people_list[k][i]]
        name = [name for name in name if ((len(name)<3) & (name[0] != people_list[k][i][0])) | (len(name)>=3)]
        name.sort(reverse=True)

        #if name != []:
        #    text = [re.sub('|'.join(name), '其他人名', news) for news in news]
        #else:
        #    text = news

        for j in index:

            mid = news[j]

            if j == 0:
                if len(news) != 1:
                    end = news[j+1]  

                    if True in [people in news[j+1] for people in x]:
                        sentences = mid
                    else:
                        sentences = mid + end
                elif len(news) == 1:
                    sentences = mid

            elif j+1 == len(news):
                start = news[j-1]

                if True in [people in news[j-1] for people in x]:
                    sentences = mid
                else:
                    sentences = start + mid

            else:
                end = news[j+1]
                start = news[j-1]                    

                if '。' in start:
                    start = ''
                elif '。' in mid:
                    end = ''                 

                if (True not in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                    sentences = start + mid + end
                elif (True not in [people in news[j-1] for people in x]) & (True in [people in news[j+1] for people in x]):
                    sentences = start + mid
                elif (True in [people in news[j-1] for people in x]) & (True not in [people in news[j+1] for people in x]):
                    sentences = mid + end
                else:
                    sentences = mid      

            AML = AML.append(pd.DataFrame([[news_ids[k] ,people_list[k][i], sentences]], columns=AML.columns))
            #break  
#把指向同一人的姓名改成一樣（陳男 -> 陳水扁），若指向多人則不改（陳男 -> 陳致中、陳水扁）
name_list = []
for ids in AML['news_id'].unique():
    full_name = [name for name in AML[(AML['news_id'] == ids)]['Name']]
    full_3name = [name for name in AML[(AML['news_id'] == ids)]['Name'] if len(name) == 3]

    a = Counter([name[0] for name in full_3name])
    keep = [k for k,v in a.items() if v == 1]
    full_3name_filter = [name for name in full_3name if name[0] in keep]
    name_dict = dict((name[0], name) for name in full_3name_filter)   # '陳' : '陳水扁'

    name_dict_2 = dict(zip([name[0:2] for name in full_3name], full_3name))  # '王音': '王音之'

    for name in full_name:
        if (name[0] in name_dict.keys()) & (len(name) == 1):
            name_list.append(name_dict.get(name[0]))
        elif (name[0] in name_dict.keys()) & (len(name) == 2) & (name[-1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                              '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                              '和', '揆', '要', '再', '董', '涉', '母',\
                                                                              '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                              '趁', '仔', '依', '氏', '父']):
            name_list.append(name_dict.get(name[0]))
        elif (name in name_dict_2.keys()) & (len(name) == 2):
            name_list.append(name_dict_2.get(name))
        else:
            name_list.append(name)


In [80]:
#排除重複資料、排除一字、兩字簡稱、兩字三字四字姓不在姓氏表中的人
AML['Name'] = name_list
AML = AML.drop_duplicates()
AML = AML[AML['Name'].apply(lambda x: (len(x) > 1) )]
AML = AML[~AML['Name'].apply(lambda x: (len(x) == 2) & (x[1] in ['男', '嫌', '婦', '夫', '某', '女', '妻',\
                                                                 '員', '稱', '家', '哥', '媽', '生', '處',\
                                                                 '和', '揆', '要', '再', '董', '涉', '母',\
                                                                 '辱', '公', '少', '為', '指', '翁', '粉',\
                                                                 '趁', '仔', '依', '氏', '父']))]

In [81]:
input_id, segment_id, mask_input = encoded(tokenizer, AML, maxlen=maxlen_sentences)
prediction = model2.predict([input_id, segment_id, mask_input])
AML['raw_prediction'] = np.round(prediction, 3)

In [82]:
#name_count = AML[['news_id', 'Name']].groupby(['news_id'])['Name'].value_counts()
#name_count = name_count[name_count == 1].reset_index(name="count").drop(['count'], axis=1)

In [83]:
# 滿足mask條件的返回
# AML['raw_prediction'] = AML['raw_prediction'].mask((AML['Name'].isin(name_count.Name)) & (AML['news_id'].isin(name_count.news_id)), AML['raw_prediction']/2)

In [84]:
AML['prediction'] = AML['raw_prediction'].apply(lambda x: 0 if x < 0.3 else 1)
AML = AML.groupby(['news_id', 'Name'])['raw_prediction', 'prediction'].max().reset_index()
AML = AML[AML['prediction'] == 1]
AMLName = AML.groupby(['news_id','prediction'])['Name'].apply(list).reset_index()
AMLPrediction = AML.groupby(['news_id'])['raw_prediction'].apply(list).apply(lambda x: np.round(x,2)).reset_index()
AMLloss = pd.merge(AMLName, AMLPrediction, how='left', on = 'news_id')
AML = pd.merge(test, AMLloss, how='left', on='news_id').drop(['content', 'prediction_y'], axis=1)

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [85]:
test_prediction = pd.merge(test, AML[['news_id', 'Name']], on='news_id', how='left')
test_prediction['Name'] = test_prediction['Name'].apply(lambda x: x if isinstance(x, list) else [])
test_prediction['text_prediction'] = test_prediction['Name'].apply(lambda x: 0 if x == [] else 1)
test_prediction = test_prediction.drop(['content'],axis = 1)
test_prediction.columns = ['news_id', 'name', 'label', 'AML_prediction', 'Name_prediction', 'text_prediction']

aml = pd.merge(test[['news_id', 'content']], test_prediction, how='left', on='news_id')
compare = pd.merge(aml, test_ner[['news_id', 'people_list']], on='news_id', how='left')
compare['people_list'].name = 'ner_namelist'
compare.to_csv('預測結果mytest.csv', encoding='utf_8_sig')

In [86]:
compare

,news_id,content,name,label,AML_prediction,Name_prediction,text_prediction,people_list
0,9999,為落實洗錢防制，有外商銀行「拒絕」立法委員等重要政治人物開戶。對此，金管會主委顧立雄昨表示，...,[],0,0.0,[],0,NaN
1,10000,瑞士調查局要求台灣外交部協助調查陳水扁家人黃睿靚，吳淑珍等人洗錢案情。早前陳水扁於8月14日...,"[吳淑珍, 陳水扁, 陳致中, 黃睿靚]",1,1.0,"[吳淑珍, 陳水扁, 陳致中, 黃睿靚]",1,"[陳致中, 吳淑珍, 陳水扁, 洪秀柱, 黃睿靚]"
2,10001,金管會今天上午公佈，常務副主委一職，由銀行局邱淑貞陞任，至於邱淑貞所遺銀行局局長，則由該局副...,[],0,0.0,[],0,NaN
3,10002,男子葉松茂誆稱行善，與台北市長郝龍斌等政治人物合照後，再拿照片詐騙民眾投資澳門威尼斯人賭場，...,"[葉松茂, 葉明宗, 葉晉展]",1,1.0,"[葉明宗, 葉晉展, 葉松茂, 郝龍斌]",1,"[葉晉展, 葉明宗, 郝龍斌, 葉, 葉松茂, 葉某]"
4,10003,有「情歌王子」之稱的台語歌手王建傑，投資髮廊，日前卻驚爆被髮廊負責人捲款逃逸，損失不少。據了...,[游登洲],1,1.0,"[游登洲, 王建傑]",1,"[游登洲, 游, 江蕙, 王建傑]"
5,10004,債留台灣700億元、十大通緝犯之一的前東帝士總裁陳由豪，2年前因投資菲律賓11兆元聲名大噪。...,"[陳由豪, 繆竹怡, 孫道存, 胡洪九]",1,1.0,"[孫道存, 繆竹怡, 胡洪九, 陳由豪]",1,"[胡洪九, 陳由豪, 杜特蒂, 繆竹怡, 陳, 孫道存, 繆]"
6,10005,金管會今天下午舉行新、卸任主委交接典禮，由政務委員行政院羅秉成監交。新任主委黃天牧致詞時表示...,[],0,0.0,[],0,NaN
7,10006,網通品牌廠友訊今日召開股東會，全面改選董事，由市場派台鋼集團取得過半席次，拿下3席董事與3席...,[],0,0.0,[],0,NaN
8,10007,2007年力霸集團創辦人王又曾掏空力霸集團案爆發，案情喧騰一時，成為國內最大宗的經濟犯罪，掏...,"[王又曾, 王金世英, 王令可, 王令麟]",1,1.0,"[王令可, 王令甫, 王令興, 王令麟, 王又曾, 王金世英]",1,"[王金世英, 王, 王又曾, 王令可, 王令麟, 王令甫, 王令興]"
9,10008,政治人物求「斗內」，小額募款不僅可造成話題，更是人氣的重要指標，而最成功的莫過於民進黨201...,[],0,0.0,[],0,NaN


In [ ]:
compare[(compare['text_prediction'] == 1) & (compare['AML_prediction'] == 1) ]

In [ ]:
compare[compare['label'] != compare['AML_prediction'] ]

In [ ]:
len(compare[(compare['label'] == 0) & (compare['AML_prediction'] == 0) ])

In [49]:
pd.set_option('display.max_rows', 1000)

In [ ]:
compare.to_csv('預測結果_ver_da.csv', encoding='utf_8_sig')

In [ ]:
compare[compare['news_id']==10024]